Import required Libraries 

In [3]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) 

Using TensorFlow backend.


In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
batch_size = 64  # Batch size for training.
epochs = 32  # Number of epochs to train for.
latent_dim = 512  # Latent dimensionality of the encoding space.
num_samples =1500 # Number of samples to train on.
# Path to the data txt file on disk.
train_eng_path = 'trainen.txt'
train_tel_path = 'trainte'

In [142]:
lines_en = pd.read_csv('trainen.txt',encoding='utf8', sep='delimiter', names=['entext'],header = None)
lines_en = lines_en[0:num_samples]
input_texts=len(lines_en)
input_texts
lines_en.head(5)

,entext
0,Any drugs in here?
1,"You're Meat Loaf, or you're made of meat loaf?"
2,My father fell off a cliff chasing a wool-blin...
3,"- NO, HE'S NOT MY DAD."
4,I told her we rest on Sundays.


In [7]:
lines_tel = pd.read_csv('trainte.txt',encoding='utf8', sep='delimiter',names=['teltext'], header=None)
#print(lines_tel)
lines_tel = lines_tel[0:num_samples]
input_texts=len(lines_tel)
input_texts

1500

 # preprocess the data

In [143]:
lines_en = lines_en["entext"].str.lower()
lines_en.head(5)


0                                   any drugs in here?
1       you're meat loaf, or you're made of meat loaf?
2    my father fell off a cliff chasing a wool-blin...
3                               - no, he's not my dad.
4                       i told her we rest on sundays.
Name: entext, dtype: object

In [140]:
lines_tel = lines_tel["teltext"].str.lower()
lines_tel.tail(5)


1495                              మేము కుడి చేసియున్నారు.
1496    గమనిక వ్యాపార చేయడం మా మార్గం ప్రశ్నించారు చేయ...
1497                      నేను యో 'గాడిద జంప్ కోరుకోలేదు.
1498                            - మీరు ప్రమాదంలో ఉన్నాయి.
1499                                   ఇదిటేలర్ఇక్కడఉంది.
Name: teltext, dtype: object

In [10]:
all_eng_words=set()
for eng in lines_en:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)
    
all_tel_words=set()
for tel in lines_tel:
    for word in tel.split():
        if word not in all_tel_words:
            all_tel_words.add(word)

In [11]:
len(all_eng_words), len(all_tel_words)

(2599, 2907)

In [136]:
print(all_tel_words)

['!', '"', '"#', '".', '"womany"', '"అడవి', '"అవర్స్', '"ఆమెన్."', '"ఉరితీయువాడు."', '"ఊ...', '"ఏమైంది?",', '"ఐ', '"ఐస్', '"ఒపేరా"', '"గ్రేటర్', '"జుడిత్', '"నాకు', '"పదం"?', '"యో,', '"లీవ్"?', '"వామ్!"', '"సర్."', '$', "'.", "'baloo", "'on", "'ఆల్!", "'ఉన్నాయి.", "'ఎందుకంటే,", "'ఒక", "'గాడిద", "'గోన్", "'ఘనతకు", "'చేతులు", "'నిద్ర", "'నేను", "'బ్రేక్.", "'మీసము", "'లాగ", "'వణుకు", "'వరుసఅసహజఉంది", '(allrepeating)', '(allఅరవటం)', '(car', '(door', '(gagging,', '(groans)', '(machine', '(man', '(menఅరవటం)', '(mouthing)', '(murmuring)', '(music', '(musicఇయర్ఫోన్స్ప్లే)', '(rapidతీవ్రతతోధ్వనిచేస్తాయి)', '(rock', '(shushing)', '(squealing', '(sunnyవైపుఉంచడంలో', '(thunder', '(tires', '(tv', '(valentine', '(zipper', '(అమ్మాయి', '(అరుపులతో)', '(అరుస్తుంటారు)', '(ఆగారు', '(ఆడమ్', '(ఇంజిన్', '(ఇంజిన్స్టార్ట్అవుతుంది)', '(ఈల)', '(ఊళ)', '(ఏడ్పులు', '(కొనసాగేదగ్గుకు)', '(కొరియన్మాట్లాడే)', '(క్లాసిక్', '(గంటనువాయించటం)', '(గుసగుసలాడుట', '(గొర్రెలు', '(తలుపు', '(తలుపుముగిసిన)', '(తలుపులు', '(తేలిగ్

In [12]:
length_en_list=[]
for le in lines_en:
    length_en_list.append(len(le.split(' ')))

length_tel_list=[]
for lt in lines_tel:
    length_tel_list.append(len(lt.split(' ')))


In [13]:
print(np.max(length_en_list),np.max(length_tel_list))

36 18


In [14]:
all_eng_words = sorted(list(all_eng_words))
all_tel_words = sorted(list(all_tel_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_tel_words)
max_encoder_seq_length = np.max(length_en_list)
max_decoder_seq_length = np.max(length_tel_list)

print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)


Number of unique input tokens: 2599
Number of unique output tokens: 2907
Max sequence length for inputs: 36
Max sequence length for outputs: 18


In [83]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(all_eng_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(all_tel_words)])

print(len(input_token_index))
print(len(target_token_index))

2599
2907


In [119]:
encoder_input_data = np.zeros(
    (num_samples,max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (num_samples, max_decoder_seq_length+1),
    dtype='float32')
decoder_target_data = np.zeros(
    (num_samples, max_decoder_seq_length+1, num_decoder_tokens),
    dtype='float32')

print(encoder_input_data.shape)
print(decoder_target_data.shape)

(1500, 36)
(1500, 19, 2907)


In [120]:
for i, (input_text, target_text) in enumerate(zip(lines_en, lines_tel)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.

In [121]:
embedding_size = 50
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model
encoder_inputs = Input(shape=(None,))

en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)

encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex= dex(decoder_inputs)

decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [122]:
print(encoder_inputs)

Tensor("input_17:0", shape=(?, ?), dtype=float32)


In [144]:
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=200,
          validation_split=0.05)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           (None, None)         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, None, 50)     129950      input_17[0][0]                   
__________________________________________________________________________________________________
embedding_14 (Embedding)        (None, None, 50)     145350      input_18[0][0]                   
__________________________________________________________________________________________________
lstm_15 (L

Epoch 46/200
1425/1425 [==============================] - 5s 4ms/step - loss: 0.9643 - acc: 0.0086 - val_loss: 1.9022 - val_acc: 0.0035
Epoch 47/200
1425/1425 [==============================] - 5s 4ms/step - loss: 0.9592 - acc: 0.0085 - val_loss: 1.9220 - val_acc: 0.0028
Epoch 48/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.9567 - acc: 0.0094 - val_loss: 1.9092 - val_acc: 0.0028
Epoch 49/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.9524 - acc: 0.0089 - val_loss: 1.9124 - val_acc: 0.0021
Epoch 50/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.9474 - acc: 0.0099 - val_loss: 1.9066 - val_acc: 0.0028
Epoch 51/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.9450 - acc: 0.0093 - val_loss: 1.9148 - val_acc: 0.0028
Epoch 52/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.9394 - acc: 0.0103 - val_loss: 1.9180 - val_acc: 0.0028
Epoch 53/200
1425/1425 [========================

Epoch 166/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.5607 - acc: 0.0709 - val_loss: 2.0751 - val_acc: 0.0042
Epoch 167/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.5586 - acc: 0.0713 - val_loss: 2.0737 - val_acc: 0.0042
Epoch 168/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.5548 - acc: 0.0719 - val_loss: 2.0776 - val_acc: 0.0021
Epoch 169/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.5528 - acc: 0.0730 - val_loss: 2.0721 - val_acc: 0.0028
Epoch 170/200
1425/1425 [==============================] - 5s 4ms/step - loss: 0.5494 - acc: 0.0732 - val_loss: 2.0797 - val_acc: 0.0028
Epoch 171/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.5472 - acc: 0.0734 - val_loss: 2.0768 - val_acc: 0.0035
Epoch 172/200
1425/1425 [==============================] - 5s 3ms/step - loss: 0.5441 - acc: 0.0745 - val_loss: 2.0781 - val_acc: 0.0028
Epoch 173/200
1425/1425 [================

In [145]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, None)              0         
_________________________________________________________________
embedding_13 (Embedding)     (None, None, 50)          129950    
_________________________________________________________________
lstm_15 (LSTM)               [(None, 50), (None, 50),  20200     
Total params: 150,150
Trainable params: 150,150
Non-trainable params: 0
_________________________________________________________________


In [146]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [147]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [151]:
for seq_index in range(1400,1500):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', lines_en[seq_index: seq_index + 1])
     #print('target sentence:', lines_tel[seq_index: seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: 1400    you grew up below the mason-dixon.
Name: entext, dtype: object
Decoded sentence:  నేను ఈ ఉంటుంది చేస్తున్నాను. గొన్న వద్ద. వద్ద. గొన్న
-
Input sentence: 1401    goodbye, loser.
Name: entext, dtype: object
Decoded sentence:  నేను ఈ ఉంటుంది చేస్తున్నాను. గొన్న వద్ద. వద్ద. గొన్న
-
Input sentence: 1402    but i graduated college and reality hit.
Name: entext, dtype: object
Decoded sentence:  మీరు ఈ పొందుటకు విసిరే జరిగినది. ఇష్టం . చేస్తుంది! చేశారు
-
Input sentence: 1403    (scooter approaching)
Name: entext, dtype: object
Decoded sentence:  నేను ఈ ఉంటుంది చేస్తున్నాను. గొన్న వద్ద. వద్ద. గొన్న
-
Input sentence: 1404    that's all i'm saying.
Name: entext, dtype: object
Decoded sentence:  మీరు ఈ ఉంటుంది గొన్న వద్ద. గొన్న వద్ద. వద్ద. గొన్న వద్ద.
-
Input sentence: 1405    uh-uh.
Name: entext, dtype: object
Decoded sentence:  నేను ఈ ఉంటుంది చేస్తున్నాను. గొన్న వద్ద. వద్ద. గొన్న
-
Input sentence: 1406    you okay to stay back and watch the fort tonight?
Name: entext, 

Decoded sentence:  మీరు ఈ డబ్బు కలిగి. చెప్పారు. వద్ద. అనుకుంటున్నారా. గొన్న
-
Input sentence: 1457    please, baby, come back.
Name: entext, dtype: object
Decoded sentence:  మీరు ఈ పొందుటకు వెళ్ళిపోయాడు క్షణం ఇబ్బంది తెలిసిపోయింది
-
Input sentence: 1458    i miss it.
Name: entext, dtype: object
Decoded sentence:  మీరు ఈ ప్రేమ. విషయం లేదు. అనుకుంటున్నారా. అనుకుంటున్నారా.
-
Input sentence: 1459    why don't we go
Name: entext, dtype: object
Decoded sentence:  మీరు ఈ అలసిపోతుంది పొందుతున్నాను. ఎప్పుడూ. ఎప్పుడూ. ఇష్టం
-
Input sentence: 1460    [ bubbling ]
Name: entext, dtype: object
Decoded sentence:  నేను డబ్బు చేస్తున్నాను. చేస్తున్నాను. చేస్తున్నాను.
-
Input sentence: 1461    she has no ribs.
Name: entext, dtype: object
Decoded sentence:  నేను సంతృప్తి చేస్తున్నాను. చేస్తున్నాను. గొన్న వద్ద.
-
Input sentence: 1462    in the day, as memories.
Name: entext, dtype: object
Decoded sentence:  నేను సంతృప్తి చేస్తున్నాను. చేస్తున్నాను. గొన్న వద్ద.
-
Input sentence: 1463    i drink it for the